# Задание 10

## Описание

Применить непараметрические критерии однородности

## Подготовка

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import (
    norm,
    chi2,
    ranksums,
    wilcoxon,
    friedmanchisquare,
    kruskal,
    median_test,
)

In [2]:
# данные о наблюдениях за массой тела до и после диеты у 10 индивидов
body_mass_before = np.array([68, 80, 92, 81, 70, 79, 78, 66, 57, 76])
body_mass_after = np.array([60, 84, 87, 79, 74, 71, 72, 67, 57, 70])

In [3]:
# время решения контрольных задач (в секундах) одиннадцатью учащимися до и после упражнений по устному счету
time_to_solve_before = np.array([87, 61, 98, 90, 93, 74, 83, 72, 81, 75, 83])
time_to_solve_after = np.array([50, 45, 79, 90, 88, 65, 52, 79, 84, 61, 52])

In [4]:
# просто матрица со слайда
presentation_matrix = np.array([
    [52, 45, 38],
    [63, 79, 50],
    [45, 57, 39],
    [53, 51, 43],
    [47, 50, 56],
    [62, 72, 49],
    [49, 52, 40],
])

In [5]:
alcoholics = pd.read_csv('data_big.csv', index_col=0)

In [6]:
addicts = pd.read_excel('addicts0.xls', sheet_name=1)

## Решение

### Критерий Вилкоксона для зависимых выборок

In [7]:
# 'greater', т.к. предполагаем, что вес уменьшится после диеты
print(wilcoxon(body_mass_before, body_mass_after, mode='approx', alternative='greater')) 

WilcoxonResult(statistic=37.0, pvalue=0.042503047233143215)


D:\Utilities\Anaconda\envs\cm\lib\site-packages\scipy\stats\_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


0.04 < 0.05 => есть основания полагать, что после диеты вес уменьшается

In [8]:
# 'greater', т.к. предполагаем, что время решения уменьшится после упражнений
print(wilcoxon(time_to_solve_before, time_to_solve_after, mode='approx', alternative='greater'))

WilcoxonResult(statistic=51.0, pvalue=0.00826724633396133)


0.008 << 0.05 => есть серьезные основания полагать, что после упражнений время на решение уменьшается

In [9]:
depressed_1 = alcoholics['depressed.mood.1'].to_numpy()
depressed_2 = alcoholics['depressed.mood.2'].to_numpy()

print(depressed_1.mean())
print(depressed_2.mean())

# 'two-sided', т.к. заранее точно нельзя сказать, как изменится уровень депрессии, ищем любую связь
print(wilcoxon(depressed_1, depressed_2, mode='approx', alternative='two-sided'))

1.1764705882352942
0.9705882352941176
WilcoxonResult(statistic=5.0, pvalue=0.019630657257290667)


0.2 < 0.05 => есть основания полагать, что наблюдается изменение. С учетом средних значений, вероятно, депрессивность уменьшается

### Критерий Фридмана для нескольких зависимых выборок

In [10]:
print(friedmanchisquare(*presentation_matrix.T))

FriedmanchisquareResult(statistic=6.0, pvalue=0.04978706836786395)


0.498 почти равно 0.05 => есть слабые основания полагать, что есть статистически значимое отличие между медианами множеств

In [11]:
depressed_data = alcoholics[['depressed.mood.1', 'depressed.mood.3', 'depressed.mood.9']].dropna().to_numpy()

print(depressed_data[:, 0].mean())
print(depressed_data[:, 1].mean())
print(depressed_data[:, 2].mean())
print(friedmanchisquare(*depressed_data.T))

1.1818181818181819
0.6363636363636364
0.09090909090909091
FriedmanchisquareResult(statistic=31.52380952380952, pvalue=1.427880088929277e-07)


1.4e-7 << 0.05 => есть серьезные основания полагать, что есть статистически значимое отличие между медианами множеств

### Критерий Краскела-Уоллиса для нескольких независимых выборок

In [12]:
eight_classes_drug, middle_school_drug, incomplete_higher_drug, higher_drug = (
    addicts[addicts.educat == 1].asi4_dr.dropna().to_numpy(),
    addicts[addicts.educat == 2].asi4_dr.dropna().to_numpy(),
    addicts[addicts.educat == 3].asi4_dr.dropna().to_numpy(),
    addicts[addicts.educat == 4].asi4_dr.dropna().to_numpy(),
)

print(kruskal(eight_classes_drug, middle_school_drug, incomplete_higher_drug, higher_drug))

KruskalResult(statistic=0.267666570559128, pvalue=0.9659902133618656)


0.966 >> 0.05 => образование не имеет статически значимого влияния на параметр "употребление наркотиков"

In [13]:
eight_classes_social, middle_school_social, incomplete_higher_social, higher_social = (
    addicts[addicts.educat == 1].asi2_emp.dropna().to_numpy(),
    addicts[addicts.educat == 2].asi2_emp.dropna().to_numpy(),
    addicts[addicts.educat == 3].asi2_emp.dropna().to_numpy(),
    addicts[addicts.educat == 4].asi2_emp.dropna().to_numpy(),
)

print(kruskal(eight_classes_social, middle_school_social, incomplete_higher_social, higher_social))

KruskalResult(statistic=8.921122877272833, pvalue=0.0303581927931991)


0.03 < 0.05 =>  образование имеет статически значимое влияния на социальный статус

### Медианный тест для нескольких независимых выборок

In [14]:
statistic, pvalue, median, table = median_test(eight_classes_drug, middle_school_drug, incomplete_higher_drug, higher_drug)

print(f"statistic: {statistic}")
print(f"p-value: {pvalue}")
print(f"median: {median}")
print(table)

statistic: 1.2381145594605654
p-value: 0.7438781224488737
median: 0.3
[[  2  42   5   2]
 [ 19 180  23   7]]


0.74 >> 0.05 => образование не имеет статически значимого влияния на параметр "употребление наркотиков"

In [15]:
statistic, pvalue, median, table = median_test(eight_classes_social, middle_school_social, incomplete_higher_social, higher_social)

print(f"statistic: {statistic}")
print(f"p-value: {pvalue}")
print(f"median: {median}")
print(table)

statistic: 6.753860518566401
p-value: 0.08017092006384198
median: 0.915
[[ 12 116   9   2]
 [  9 105  18   7]]


0.08 > 0.05 =>  образование может влияният на социальный статус